In [14]:
# Clear memory
%reset Out

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Flushing output cache (5 entries)


In [15]:
# Import libraries
import math
import pandas as pd
import numpy as np
import gensim.models as gsm
import multiprocessing as mp
import datetime as dt

from scipy.spatial.distance import cosine

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn.metrics import log_loss

In [16]:
# Import data
train_lookup_df = pd.read_pickle('./pickles.gi/train_lookup_df.pkl')
questions_df = pd.read_pickle('./pickles.gi/questions_with_tokens_df.pkl')

In [17]:
# Get initial number of rows for processing validation
num_rows_questions_df_1 = questions_df.shape[0]

In [18]:
# Get sentences for vectorization
questions_df['preproc_question'] = [' '.join(token_list) for token_list in questions_df.token_list]

In [19]:
# Reshape dataframe from stacked to wide format (pivot on pair ID)
questions_df = questions_df.merge(questions_df, on=['test', 'pid'], how='inner', suffixes=('_1', '_2'))
questions_df = questions_df[(questions_df['qid_1'] != questions_df['qid_2']) & (questions_df['qid_1'] < questions_df['qid_2'])]
questions_df.reset_index(drop=True, inplace=True)
questions_df.sort_values(by=['test', 'pid'], inplace=True)

In [20]:
# Validate that the number of rows * 2 in current DF is equal to imported DF
num_rows_questions_df_2 = questions_df.shape[0]
assert num_rows_questions_df_1 == num_rows_questions_df_2 * 2, "Number of rows in questions_df is not consistent with original import"

In [21]:
# Get a term vector for each question
vectorizer = TfidfVectorizer()
tfidf_vector_1 = vectorizer.fit_transform(questions_df.preproc_question_1)
tfidf_vector_2 = vectorizer.fit_transform(questions_df.preproc_question_2)

In [37]:
tfidf_vector_1[245245]

<1x99949 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [38]:
tfidf_vector_2[454]

<1x100084 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [ ]:
questions_df['tfidf_cos_sim'] = questions_df.apply(lambda x: (1 - cosine(x['tfidf_vect_1'].toarray(), x['tfidf_vect_2'].toarray())), axis=1)
questions_df.to_pickle('./pickles.gi/questions_df_tfidf_cosine_similarity')

In [30]:
cosine(tfidf_vector_1[0].toarray(), tfidf_vector_1[0].toarray())

-2.2204460492503131e-16